<h4>Mahdi Abbariki</h4>
<h5>983212001</h5>

first, we import packages that we want to use

In [675]:
import numpy as np

configuration of this algorithm is done in a <b>File</b> named geneticConfig <br>
read the config file and populate the initial variables

<b>keep the line orders as it is</b>
<ul>
    <li>line 1 : number of population</li>
    <li>line 2 : minimum of search range</li>
    <li>line 3 : maximum of search range</li>
    <li>line 4 : mutation probability</li>
    <li>line 5 : crossover probability</li>
    <li>line 6 : crossover type (1 for 1-point, 2 for uniform)</li>
    <li>line 7 : max iterations of the algorithm</li>
</ul>

In [676]:
config = open('geneticConfig', 'r').readlines()
# number of initial chromosomes
maxPopulation = int(config[0].strip())

# min range of solution
minRange = int(config[1].strip())
# max range of solution
maxRange = int(config[2].strip())
# mutation probability
pm = float(config[3].strip())
# crossover probability
pc = float(config[4].strip())
# crossover type
crossoverType = int(config[5].strip())
# max iteration
maxIteration = int(config[6].strip())

# bit needed
maxBitNeeded = maxRange.bit_length()
# population must be even
if(maxPopulation%2!=0): maxPopulation = maxPopulation+1

create a method to compute the function based on input x <br>
another method to do SoftMax on the array results

In [677]:
def computeFunction(x):
    return np.negative(np.power(x,2)) + (6 * x)


def softmax(x):
    return (np.exp(x)/np.exp(x).sum())


then, we initialize population<br>
generate the initial population by generating random binary numbers inside the range

In [678]:
def generateInitialPopulation():
    population = [] 
    for index in range(maxPopulation):
        str = '{0:0{bitLen}b}'.format(np.random.randint(low=minRange, high=maxRange), bitLen=maxBitNeeded)
        population.append(list(str))
    return population

After preparing the population, Based on the fitness function, we are going to select the best individuals within the current population as parents for mating. 
<br>
normalize the fitnesses between 0 to 1 to generate roulette wheel probabilities

In [679]:
def computeFitnesses(population):
    fitnesses = np.zeros(len(population))
    for index,item in enumerate(population):
        x = int(''.join(item),2)
        fitnesses[index] = computeFunction(x)
    # normalize fitnesses for probability of selecting each parent
    fitnesses = (fitnesses - np.min(fitnesses)) / \
        (np.max(fitnesses) - np.min(fitnesses) + np.finfo(float).eps)
    return fitnesses

based on fitnesses array, now we can select best parents for mating pool
<br>
best parents is generated using <b>roulette wheel</b>

In [680]:
def selectMatingPool(population, fitnesses):
    parents = []
    probability = []
    s = 0
    fitnessesSoftMax = softmax(fitnesses)
    for k in range(len(fitnessesSoftMax)):
        s = s + fitnessesSoftMax[k]
        probability.append(s)
    for k in range(maxPopulation):
        rand = np.random.uniform(low=0,high=1)
        rouletteWheelIndex = [n for n, i in enumerate(probability) if i > rand][0]
        parents.append(population[rouletteWheelIndex])
    return parents

after selecting mating pool from population, now we can generate next generation using crossover with the selected type in config

In [681]:
def onePointCrossover(parents):
    offspring = [[] for _ in range(len(parents))]
    for k in range(len(parents)):
        randPc = np.random.uniform(low=0,high=1)
        firstParent = k % len(parents)
        secondParent = (k+1) % len(parents)
        if(randPc <= pc):
            point = np.random.randint(low=0,high=maxBitNeeded)
            offspring[k][0:point] = parents[firstParent][0:point]
            offspring[k][point:] = parents[secondParent][point:]
        else:
            offspring[k] = parents[firstParent]
    return offspring


def uniformCrossover(parents):
    offspring = [['0' for _ in range(maxBitNeeded)] for _ in range(len(parents))]
    for k in range(len(parents)):
        randPc = np.random.uniform(low=0, high=1)
        firstParent = k % len(parents)
        secondParent = (k+1) % len(parents)
        if (randPc <= pc):
            for j in range(len(parents[k])):
                rand = np.random.uniform(low=0,high=1)
                if(rand <= 0.5):
                    offspring[k][j] = parents[firstParent][j]
                else:
                    offspring[k][j] = parents[secondParent][j]
        else:
            offspring[k] = parents[firstParent]
    return offspring



def doCrossOver(parents):
    if(type == 1):
        return onePointCrossover(parents=parents)
    else:
        return uniformCrossover(parents=parents)

now we can perform mutation
<br>
mutation is done generating random number between 0 to 1 for each bit of children

In [682]:
def mutation(offspring):
    for k in range(len(offspring)):
        for j in range(len(offspring[k])):
            randPm = np.random.uniform(low=0, high=1)
            if (randPm <= pm):
                if (offspring[k][j] == '0'):
                    offspring[k][j] = '1'
                else:
                    offspring[k][j] = '0'
    return offspring

now we have all methods that we need to build the genetics algorithm, so let's build it 
<br>
print the final result

In [683]:
population = generateInitialPopulation()
for generation in range(maxIteration):
    fitness = computeFitnesses(population=population)
    parents = selectMatingPool(population=population,fitnesses=fitness)
    offspring = doCrossOver(parents=parents)
    offspring = mutation(offspring=offspring)

    population = offspring

bestFitness = computeFitnesses(population=population)
bestGeneIndex = np.where(bestFitness == np.max(bestFitness))
x = int(''.join(population[bestGeneIndex[0][0]]), 2)
print("best answer founded is:", x)
print("equation value with this value is:",computeFunction(x))


best answer founded is: 3
equation value with this value is: 9
